In [ ]:
import cv2
import mediapipe as mp
import numpy as np

mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose

# 랜드마크 이름 리스트
POSE_LANDMARKS = [
    'NOSE', 'LEFT_EYE_INNER', 'LEFT_EYE', 'LEFT_EYE_OUTER', 'RIGHT_EYE_INNER', 'RIGHT_EYE', 'RIGHT_EYE_OUTER',
    'LEFT_EAR', 'RIGHT_EAR', 'MOUTH_LEFT', 'MOUTH_RIGHT', 'LEFT_SHOULDER', 'RIGHT_SHOULDER', 'LEFT_ELBOW',
    'RIGHT_ELBOW', 'LEFT_WRIST', 'RIGHT_WRIST', 'LEFT_PINKY', 'RIGHT_PINKY', 'LEFT_INDEX', 'RIGHT_INDEX',
    'LEFT_THUMB', 'RIGHT_THUMB', 'LEFT_HIP', 'RIGHT_HIP', 'LEFT_KNEE', 'RIGHT_KNEE', 'LEFT_ANKLE', 'RIGHT_ANKLE',
    'LEFT_HEEL', 'RIGHT_HEEL', 'LEFT_FOOT_INDEX', 'RIGHT_FOOT_INDEX'
]

cap = cv2.VideoCapture(0)

with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    while cap.isOpened():
        success, image = cap.read()
        if not success:
            print("웹캠에서 프레임을 읽을 수 없습니다.")
            continue

        image.flags.writeable = False
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        results = pose.process(image)

        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
        if results.pose_landmarks:
            mp_drawing.draw_landmarks(
                image,
                results.pose_landmarks,
                mp_pose.POSE_CONNECTIONS,
                mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=2),
                mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
            )
            
          
            for idx, landmark in enumerate(results.pose_landmarks.landmark):
                h, w, c = image.shape
                cx, cy = int(landmark.x * w), int(landmark.y * h)
                cv2.circle(image, (cx, cy), 5, (255, 0, 0), cv2.FILLED)
                cv2.putText(image, f'{idx}: {POSE_LANDMARKS[idx]}', (cx + 10, cy),
                            cv2.FONT_HERSHEY_PLAIN, 0.8, (0, 0, 255), 1)

        cv2.imshow('MediaPipe Pose with Landmarks', image)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
        
cap.release()
cv2.destroyAllWindows()

In [11]:
import cv2
import mediapipe as mp
import numpy as np

mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose

# 선택된 랜드마크 리스트
SELECTED_LANDMARKS = [
    'NOSE', 'LEFT_EYE_INNER', 'LEFT_EYE', 'LEFT_EYE_OUTER', 'RIGHT_EYE_INNER', 'RIGHT_EYE', 'RIGHT_EYE_OUTER',
    'LEFT_EAR', 'RIGHT_EAR', 'MOUTH_LEFT', 'MOUTH_RIGHT', 'LEFT_SHOULDER', 'RIGHT_SHOULDER', 'LEFT_ELBOW',
    'RIGHT_ELBOW', 'LEFT_WRIST', 'RIGHT_WRIST', 'LEFT_PINKY', 'RIGHT_PINKY', 'LEFT_INDEX', 'RIGHT_INDEX',
    'LEFT_THUMB', 'RIGHT_THUMB', 'LEFT_HIP', 'RIGHT_HIP', 'LEFT_KNEE', 'RIGHT_KNEE', 'LEFT_ANKLE', 'RIGHT_ANKLE',
    'LEFT_HEEL', 'RIGHT_HEEL', 'LEFT_FOOT_INDEX', 'RIGHT_FOOT_INDEX'
]

#1~33까지 인덱스 중에서 위에서 고른 랜드마크의 고유 인덱스를 얻어서 S_L_I 변수에 저장
SELECTED_LANDMARK_INDICES = [mp_pose.PoseLandmark[landmark].value for landmark in SELECTED_LANDMARKS]
#비디오 캡쳐
video_path="비디오 경로"
cap = cv2.VideoCapture(0)
cap_video=cv2.VideoCapture(video_path)

with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose: 
    #min_detection_confidence와 min_tracking_confidence는 각각 최소 감지 신뢰도와 최소 추적 신뢰도를 설정하고 Pose객체를 초기화
    while cap.isOpened(): #웹캠의 cap이 열려있는동안 반복실행
        success, image = cap.read() #cap의 프레임이 제대로 들어갔는지에 대한 불리언값과 그 프레임의 이미지를 저장
        if not success: #프레임이 제대로 들어가지 않아 False가 저장되어있을경우 메세지를 출력 해주고 이어서 실행
            print("웹캠에서 프레임을 읽을 수 없습니다.")
            continue

        image.flags.writeable = False #이미지 파일을 불러올때 쓰기 권한을 False로 해제. 읽기권한만 있는 상태로 읽어들어오는것이 더 빠름.
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) #mediapipe는 RGB 형식으로 이미지를 처리하고 cv는 BGR로 처리하기때문에 mediapipe에 맞게 변환
        results = pose.process(image) #pose객체로 이미지를 처리해서 관절을 추적 이때 추적된 관절들의 정보가 results에 저장됨
        image.flags.writeable = True #이미지의 쓰기권한을 부여
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR) #pose처리가 된 이미지를 다시 cv 형식으로 바꿈

        if results.pose_landmarks:
            # 선택된 랜드마크만 그리기
            landmarks_to_draw = mp_pose.PoseLandmark._member_names_
            connections_to_draw = [
                connection for connection in mp_pose.POSE_CONNECTIONS
                if connection[0] in SELECTED_LANDMARK_INDICES and connection[1] in SELECTED_LANDMARK_INDICES
            ]

            mp_drawing.draw_landmarks(
                image,
                results.pose_landmarks,
                connections_to_draw,
                mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=2),
                mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
            )

            # 선택된 랜드마크에 대해서만 라벨 표시
            for idx, landmark in enumerate(results.pose_landmarks.landmark):
                if idx in SELECTED_LANDMARK_INDICES:
                    h, w, c = image.shape
                    cx, cy = int(landmark.x * w), int(landmark.y * h)
                    cv2.circle(image, (cx, cy), 5, (255, 0, 0), cv2.FILLED)
                    cv2.putText(image, f'{SELECTED_LANDMARKS[SELECTED_LANDMARK_INDICES.index(idx)]}', 
                                (cx + 10, cy), cv2.FONT_HERSHEY_PLAIN, 0.8, (0, 0, 255), 1)

        cv2.imshow('MediaPipe Pose with Selected Landmarks', image)
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()

AttributeError: 'NoneType' object has no attribute 'landmark'

In [1]:
import cv2
import mediapipe as mp
import numpy as np

mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose


In [8]:
mp_pose.PoseLandmark['LEFT_EYE_INNER'].value

1

In [22]:
import cv2
import mediapipe as mp
import numpy as np

mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose

SELECTED_LANDMARKS = [
    'NOSE', 'LEFT_EYE_INNER', 'LEFT_EYE', 'LEFT_EYE_OUTER', 'RIGHT_EYE_INNER', 'RIGHT_EYE', 'RIGHT_EYE_OUTER',
    'LEFT_EAR', 'RIGHT_EAR', 'MOUTH_LEFT', 'MOUTH_RIGHT', 'LEFT_SHOULDER', 'RIGHT_SHOULDER', 'LEFT_ELBOW',
    'RIGHT_ELBOW', 'LEFT_WRIST', 'RIGHT_WRIST', 'LEFT_PINKY', 'RIGHT_PINKY', 'LEFT_INDEX', 'RIGHT_INDEX',
    'LEFT_THUMB', 'RIGHT_THUMB', 'LEFT_HIP', 'RIGHT_HIP', 'LEFT_KNEE', 'RIGHT_KNEE', 'LEFT_ANKLE', 'RIGHT_ANKLE',
    'LEFT_HEEL', 'RIGHT_HEEL', 'LEFT_FOOT_INDEX', 'RIGHT_FOOT_INDEX'
]

SELECTED_LANDMARK_INDICES = [mp_pose.PoseLandmark[landmark].value for landmark in SELECTED_LANDMARKS]

def calculate_angle(a, b, c):
    a = np.array(a)
    b = np.array(b)
    c = np.array(c)
    
    ba = a - b
    bc = c - b
    
    cosine_angle = np.dot(ba, bc) / (np.linalg.norm(ba) * np.linalg.norm(bc))
    angle = np.arccos(cosine_angle)

    return np.degrees(angle)
#관절의 각도 계산 b관절을 기준으로 a관절과 c관절의 cos값을 계산. cos값이 -1~1까지 나오므로 코사인의 역함수 값을 취하면 0~180도가 나옴. cos(-1): 180 cos(0):90
#cos (0도) = 1   cos(90도) =0 cos (180도) = -1


def process_frame(frame, pose):
    frame.flags.writeable = False
    frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = pose.process(frame)
    frame.flags.writeable = True
    frame = cv2.cvtColor(frame, cv2.COLOR_RGB2BGR)

    if results.pose_landmarks:
        landmarks = results.pose_landmarks.landmark
        connections_to_draw = [
            connection for connection in mp_pose.POSE_CONNECTIONS
            if connection[0] in SELECTED_LANDMARK_INDICES and connection[1] in SELECTED_LANDMARK_INDICES
        ]
        mp_drawing.draw_landmarks(
            frame, results.pose_landmarks, connections_to_draw,
            mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=2),
            mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
        )

    return frame, results
#관절의 좌표값을 얻어서 angles리스트에 담아 cal angles함수로 보냄
def get_angles(landmarks):
    angles = []
    
    # 오른쪽 팔꿈치 각도
    right_shoulder = [landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].x,
                      landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].y]
    right_elbow = [landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].x,
                   landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].y]
    right_wrist = [landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].x,
                   landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].y]
    angle = calculate_angle(right_shoulder, right_elbow, right_wrist)
    angles.append(("Right Elbow", angle))

    # 왼쪽 팔꿈치 각도
    left_shoulder = [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x,
                     landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y]
    left_elbow = [landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].x,
                  landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].y]
    left_wrist = [landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].x,
                  landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].y]
    angle = calculate_angle(left_shoulder, left_elbow, left_wrist)
    angles.append(("Left Elbow", angle))

    # 오른쪽 무릎 각도
    right_hip = [landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].x,
                 landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].y]
    right_knee = [landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].x,
                  landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].y]
    right_ankle = [landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].x,
                   landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].y]
    angle = calculate_angle(right_hip, right_knee, right_ankle)
    angles.append(("Right Knee", angle))

    # 왼쪽 무릎 각도
    left_hip = [landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].x,
                landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].y]
    left_knee = [landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].x,
                 landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].y]
    left_ankle = [landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].x,
                  landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].y]
    angle = calculate_angle(left_hip, left_knee, left_ankle)
    angles.append(("Left Knee", angle))

    
    return angles

video_path = "fit.mp4"
cap = cv2.VideoCapture(0)
cap_video = cv2.VideoCapture(video_path)

with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    while cap.isOpened() and cap_video.isOpened():
        ret, frame = cap.read()
        ret_video, frame_video = cap_video.read()

        if not ret or not ret_video:
            print("Can't receive frame (stream end?). Exiting ...")
            break

        frame, results = process_frame(frame, pose)
        frame_video, results_video = process_frame(frame_video, pose)

        if results.pose_landmarks and results_video.pose_landmarks:
            angles_webcam = get_angles(results.pose_landmarks.landmark)
            angles_video = get_angles(results_video.pose_landmarks.landmark)
            
            angle_differences = np.array(angles_webcam) - np.array(angles_video)
            
            # 각도 차이를 화면에 표시
            cv2.putText(frame, f"Angle Diff: {angle_differences}", 
                        (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)

        # 두 영상을 나란히 표시
        combined_frame = np.hstack((frame, frame_video))
        cv2.imshow('Webcam and Video Comparison', combined_frame)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

cap.release()
cap_video.release()
cv2.destroyAllWindows()

KeyboardInterrupt: 

In [23]:
import cv2
import mediapipe as mp
import numpy as np

mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose

SELECTED_LANDMARKS = [
    'NOSE', 'LEFT_EYE_INNER', 'LEFT_EYE', 'LEFT_EYE_OUTER', 'RIGHT_EYE_INNER', 'RIGHT_EYE', 'RIGHT_EYE_OUTER',
    'LEFT_EAR', 'RIGHT_EAR', 'MOUTH_LEFT', 'MOUTH_RIGHT', 'LEFT_SHOULDER', 'RIGHT_SHOULDER', 'LEFT_ELBOW',
    'RIGHT_ELBOW', 'LEFT_WRIST', 'RIGHT_WRIST', 'LEFT_PINKY', 'RIGHT_PINKY', 'LEFT_INDEX', 'RIGHT_INDEX',
    'LEFT_THUMB', 'RIGHT_THUMB', 'LEFT_HIP', 'RIGHT_HIP', 'LEFT_KNEE', 'RIGHT_KNEE', 'LEFT_ANKLE', 'RIGHT_ANKLE',
    'LEFT_HEEL', 'RIGHT_HEEL', 'LEFT_FOOT_INDEX', 'RIGHT_FOOT_INDEX'
]

SELECTED_LANDMARK_INDICES = [mp_pose.PoseLandmark[landmark].value for landmark in SELECTED_LANDMARKS]

def calculate_angle(a, b, c):
    a = np.array(a)
    b = np.array(b)
    c = np.array(c)
    
    ba = a - b
    bc = c - b
    
    cosine_angle = np.dot(ba, bc) / (np.linalg.norm(ba) * np.linalg.norm(bc))
    angle = np.arccos(cosine_angle)

    return np.degrees(angle)
#관절의 각도 계산 b관절을 기준으로 a관절과 c관절의 cos값을 계산. cos값이 -1~1까지 나오므로 코사인의 역함수 값을 취하면 0~180도가 나옴. cos(-1): 180 cos(0):90
#cos (0도) = 1   cos(90도) =0 cos (180도) = -1


def process_frame(frame, pose):
    frame.flags.writeable = False
    frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = pose.process(frame)
    frame.flags.writeable = True
    frame = cv2.cvtColor(frame, cv2.COLOR_RGB2BGR)

    if results.pose_landmarks:
        landmarks = results.pose_landmarks.landmark
        connections_to_draw = [
            connection for connection in mp_pose.POSE_CONNECTIONS
            if connection[0] in SELECTED_LANDMARK_INDICES and connection[1] in SELECTED_LANDMARK_INDICES
        ]
        mp_drawing.draw_landmarks(
            frame, results.pose_landmarks, connections_to_draw,
            mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=2),
            mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
        )

    return frame, results
#관절의 좌표값을 얻어서 angles리스트에 담아 cal angles함수로 보냄
def get_angles(landmarks):
    angles = []
    
    # 오른쪽 팔꿈치 각도
    right_shoulder = [landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].x,
                      landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].y]
    right_elbow = [landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].x,
                   landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].y]
    right_wrist = [landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].x,
                   landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].y]
    angle = calculate_angle(right_shoulder, right_elbow, right_wrist)
    angles.append(angle)

    # 왼쪽 팔꿈치 각도
    left_shoulder = [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x,
                     landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y]
    left_elbow = [landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].x,
                  landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].y]
    left_wrist = [landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].x,
                  landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].y]
    angle = calculate_angle(left_shoulder, left_elbow, left_wrist)
    angles.append(angle)

    # 오른쪽 무릎 각도
    right_hip = [landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].x,
                 landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].y]
    right_knee = [landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].x,
                  landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].y]
    right_ankle = [landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].x,
                   landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].y]
    angle = calculate_angle(right_hip, right_knee, right_ankle)
    angles.append(angle)

    # 왼쪽 무릎 각도
    left_hip = [landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].x,
                landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].y]
    left_knee = [landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].x,
                 landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].y]
    left_ankle = [landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].x,
                  landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].y]
    angle = calculate_angle(left_hip, left_knee, left_ankle)
    angles.append(angle)

    return angles

# 메인 루프 부분
with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    while cap.isOpened() and cap_video.isOpened():
        ret, frame = cap.read()
        ret_video, frame_video = cap_video.read()

        if not ret or not ret_video:
            print("Can't receive frame (stream end?). Exiting ...")
            break

        frame, results = process_frame(frame, pose)
        frame_video, results_video = process_frame(frame_video, pose)

        if results.pose_landmarks and results_video.pose_landmarks:
            angles_webcam = get_angles(results.pose_landmarks.landmark)
            angles_video = get_angles(results_video.pose_landmarks.landmark)
            
            angle_differences = np.array(angles_webcam) - np.array(angles_video)
            
            # 각도 차이를 화면에 표시
            joint_names = ["Right Elbow", "Left Elbow", "Right Knee", "Left Knee"]
            for i, (name, diff) in enumerate(zip(joint_names, angle_differences)):
                cv2.putText(frame, f"{name}: {diff:.2f}", 
                            (10, 30 + i*30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
            
            # 각도 차이 배열 출력
            print(f"Angle differences: {angle_differences}")

        # 이미지 크기 맞추기
        height = min(frame.shape[0], frame_video.shape[0])
        width = min(frame.shape[1], frame_video.shape[1])

        frame_resized = cv2.resize(frame, (width, height))
        frame_video_resized = cv2.resize(frame_video, (width, height))

        # 두 영상을 나란히 표시
        combined_frame = np.hstack((frame_resized, frame_video_resized))
        cv2.imshow('Webcam and Video Comparison', combined_frame)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

cap.release()
cap_video.release()
cv2.destroyAllWindows()


In [19]:
print(f"Angle differences: {angle_differences}")

Angle differences: [  27.10811191  -81.69426626 -157.8726598  -143.59231235]
